In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

# player_ids = ['677950', '572233', '682998', '666150', '656896', '553993', '672515', '672695', '607054', '664983', '645444', '606466', '666971', '656976', '592626']
timeout = 25
# mlb_seasons = ['2023', '2022', '2021']
player_ids = [['dbacks', 'Alek Thomas', 'alek-thomas-677950']]


In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [ ]:
url = f"https://baseballsavant.mlb.com/savant-player/alek-thomas-677950?stats=statcast-r-hitting-mlb"
driver.get(url)

pitches = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartBreakdown"]'))
pitches.select_by_index(0)

handedness = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartHandedness"]'))
handedness.select_by_index(1)

headers_present = EC.visibility_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]/tbody/tr[1]/td[2]/span'))
WebDriverWait(driver, timeout).until(headers_present)

src = driver.page_source
parser = BeautifulSoup(src, "lxml")
table = parser.find("div", attrs = {"id": "pitchTable"})

headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers[:]]
headerlist.insert(0, "Name")
headerlist.insert(2, "Team")


In [ ]:
headerlist 

In [ ]:
for player in player_ids:
    url = f"https://baseballsavant.mlb.com/savant-player/{player[-1]}?stats=statcast-r-hitting-mlb"
    driver.get(url)

    pitches = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartBreakdown"]'))
    pitches.select_by_index(0)

    handedness = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartHandedness"]'))

    for hand in range(1, len(handedness.options)):
        handedness.select_by_index(hand)
        
        first_row_present = EC.visibility_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]'))
        WebDriverWait(driver, timeout).until(first_row_present)

        src = driver.page_source
        parser = BeautifulSoup(src, "lxml")
        table = parser.find("div", attrs = {"id": "pitchTable"})
    
        rows = table.findAll('tr')[1:]
    
        player_stats = []
        for i in range(len(rows)):
            row_cells = []
            for td in rows[i].findAll('td'):
                row_cells.append(td.getText().strip())
            row_cells.insert(0, player[1])
            row_cells.insert(2, player[0])
            player_stats.append(row_cells)
    

In [ ]:
driver.quit()

In [ ]:
player_stats

In [ ]:
rows

In [ ]:
df = pd.DataFrame(player_stats, columns=headerlist)

In [ ]:
df

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


payload = {}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101 Firefox/122.0',
  'Accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.5',
  'Accept-Encoding': 'gzip, deflate, br',
  'X-Requested-With': 'XMLHttpRequest',
  'Connection': 'keep-alive',
  'Referer': 'https://baseballsavant.mlb.com/savant-player/alek-thomas-677950?stats=statcast-r-hitting-mlb',
  'Cookie': '__cflb=02DiuJFcgbX2zpcHdRurfpjkBTD5BPySo6gPFqPfkQ556; AMCV_A65F776A5245B01B0A490D44%40AdobeOrg=1099438348%7CMCIDTS%7C19755%7CMCMID%7C08223760795725670020171971273569729942%7CMCAAMLH-1707411436%7C7%7CMCAAMB-1707411436%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1706813836s%7CNONE%7CMCAID%7CNONE%7CMCSYNCSOP%7C411-19762%7CvVersion%7C2.1.0; s_cc=true; s_sq=%5B%5BB%5D%5D; AMCVS_A65F776A5245B01B0A490D44%40AdobeOrg=1; s_vi=[CS]v1|32DDE6B6150B13BF-40000A6BE1F97D36[CE]; s_ecid=MCMID%7C08223760795725670020171971273569729942; _dd_s=rum=0&expire=1706807948264',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'TE': 'trailers'
}


players = [['dbacks', 'Alek Thomas', '677950']]
handedness =['R', 'L']

In [ ]:
for player in players:
    player_stats = []
    for hand in handedness:
        url = f"https://baseballsavant.mlb.com/player-services/statcast-pitches-breakdown?playerId={player[-1]}&position=8&hand={hand}&pitchBreakdown=pitches&timeFrame=yearly&season=&pitchType=&count=&updatePitches=true"

        r = requests.get(url, headers=headers)

        parser = BeautifulSoup(r.content, "html.parser")
        table = parser.find(id="detailedPitches")

        headersTH = table.find_all('th')
        headerlist = [h.text.strip() for h in headersTH[:]]
        headerlist.insert(0, "Name")
        headerlist.insert(2, "Team")
        headerlist.insert(3, "Hand") 
    
        rows = table.findAll('tr')[1:]
    
        for i in range(len(rows)):
            row_cells = []
            for td in rows[i].findAll('td'):
                row_cells.append(td.getText().strip())
            row_cells.insert(0, player[1])
            row_cells.insert(2, player[0])
            row_cells.insert(3, hand)
            player_stats.append(row_cells)

In [ ]:
headerlist

In [ ]:
player_stats

In [ ]:
df = pd.DataFrame(player_stats, columns=headerlist)


In [ ]:
df

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

timeout = 25

handedness =['R', 'L']
player_stats = []
headerlist = []
pitcher_stats = []
players = [['dbacks', 'Brandon Pfaadt', 'brandon-pfaadt-694297', 'Pitcher']]
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [2]:
for player in players:
    url = f"https://baseballsavant.mlb.com/savant-player/{player[2]}?stats=statcast-r-pitching-mlb"
    driver.get(url)

    detailedPitches_present = EC.visibility_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]/tbody/tr[1]/td[2]/span'))
    WebDriverWait(driver, timeout).until(detailedPitches_present)

    src = driver.page_source
    parser = BeautifulSoup(src, "lxml")

    pitchMovement = parser.find("table", attrs = {"id": "pitchMovement"})
    detailedPitches = parser.find("table", attrs = {"id": "detailedPitches"})
    runValues = parser.find("table", attrs = {"id": "runValues"})
    spinDirection = parser.find("table", attrs = {"id": "spinAxis"})
    
    if not headerlist:
        pitchMovementHeaders = pitchMovement.findAll('th')
        pitchMovementHeadersList = [h.text.strip() for h in pitchMovementHeaders[:]]

        detailedPitchesHeaders = detailedPitches.findAll('th')
        detailedPitchesHeadersList = [h.text.strip() for h in detailedPitchesHeaders[:]]

        runValuesHeaders = runValues.findAll('th')
        runValuesHeadersList = [h.text.strip() for h in runValuesHeaders[:]]

        spinDirectionHeaders = spinDirection.findAll('th')
        spinDirectionHeadersList = [h.text.strip() for h in spinDirectionHeaders[:]]
    
        pitchMovementHeadersList_2 = pitchMovementHeadersList[2:]
        pitchMovementHeadersList_2[0] = 'Name'
        headerlist.extend(pitchMovementHeadersList_2)
        headerlist.extend(detailedPitchesHeadersList[3:])
        headerlist.extend(runValuesHeadersList[3:])
        headerlist.extend(spinDirectionHeadersList[3:])

    pitchMovementRows = pitchMovement.findAll('tr')[2:]
    detailedPitchesRows = detailedPitches.findAll('tr')[1:]
    runValuesRows = runValues.findAll('tr')[1:]
    spinDirectionRows = spinDirection.findAll('tr')[1:]
    
    pitcher1_stats = []
    pitcher2_stats = []
    pitcher3_stats = []
    pitcher4_stats = []
    for i in range(len(pitchMovementRows)):
        row_cells = []
        for td in pitchMovementRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        pitcher1_stats.append(row_cells)

    for i in range(len(detailedPitchesRows)):
        row_cells = []
        for td in detailedPitchesRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        pitcher2_stats.append(row_cells)

    for i in range(len(runValuesRows)):
        row_cells = []
        for td in runValuesRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        pitcher3_stats.append(row_cells)

    for i in range(len(spinDirectionRows)):
        row_cells = []
        for td in spinDirectionRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        pitcher4_stats.append(row_cells)

    for i in range(len(pitchMovementRows)):
        for pitch in pitcher1_stats:
            if pitch[1] == pitcher2_stats[i][1]:
                pitch.extend(pitcher2_stats[i][3:])

    for i in range(len(pitchMovementRows)):
        for pitch in pitcher1_stats:
            if pitch[1] == pitcher3_stats[i][1]:
                pitch.extend(pitcher3_stats[i][3:])

    for i in range(len(pitchMovementRows)):
        for pitch in pitcher1_stats:
            if pitch[1] == pitcher4_stats[i][1]:
                pitch.extend(pitcher4_stats[i][3:])
    
    for pitch in pitcher1_stats: 
        pitch.insert(0, player[1])

driver.quit()
df = pd.DataFrame(pitcher1_stats, columns=headerlist)

In [3]:
driver.quit()

In [ ]:
for i in range(len(pitchMovementRows)):
    for pitch in pitcher1_stats:
        if pitch[1] == pitcher2_stats[i][1]:
            pitch.extend(pitcher2_stats[i][3:])

In [ ]:
for i in range(len(pitchMovementRows)):
    for pitch in pitcher1_stats:
        if pitch[1] == pitcher3_stats[i][1]:
            pitch.extend(pitcher3_stats[i][3:])

In [ ]:
for i in range(len(pitchMovementRows)):
    for pitch in pitcher1_stats:
        if pitch[1] == pitcher4_stats[i][1]:
            pitch.extend(pitcher4_stats[i][3:])

In [ ]:
for pitch in pitcher1_stats[1:]: 
    pitch.insert(0, 'Brandon')

In [4]:
pitcher1_stats

[['Brandon Pfaadt',
  '2023',
  'Sinker',
  'AZ',
  'R',
  '160',
  '93.0',
  '24.4',
  '0.9',
  '4',
  '15.5',
  '0.4',
  '3',
  '125',
  '35',
  '9.9',
  '93.0',
  '35',
  '32',
  '10',
  '6',
  '2',
  '0',
  '2',
  '7',
  '25',
  '.313',
  '.273',
  '.563',
  '.506',
  '.398',
  '.362',
  '89.7',
  '6',
  '2311',
  '6.8',
  '18.8',
  '33.3',
  '-2.0',
  '-3',
  '160',
  '9.9',
  '35',
  '.313',
  '.563',
  '.426',
  '18.8',
  '20.0',
  '33.3',
  '.273',
  '.506',
  '.361',
  '36.0',
  '93.0',
  '83',
  '17.4',
  '1:15',
  '2:15',
  '-60'],
 ['Brandon Pfaadt',
  '2023',
  'Changeup',
  'AZ',
  'R',
  '195',
  '86.7',
  '33.2',
  '0.9',
  '3',
  '15.8',
  '1.3',
  '9',
  '30',
  '165',
  '12.1',
  '86.7',
  '45',
  '44',
  '18',
  '11',
  '3',
  '1',
  '3',
  '9',
  '35',
  '.409',
  '.369',
  '.727',
  '.604',
  '.483',
  '.419',
  '91.4',
  '13',
  '1954',
  '6.9',
  '33.3',
  '17.0',
  '-4.1',
  '-8',
  '195',
  '12.1',
  '45',
  '.409',
  '.727',
  '.508',
  '33.3',
  '20.0',
  '1

In [ ]:
pitcherpp

In [5]:
df = pd.DataFrame(pitcher1_stats, columns=headerlist)

In [6]:
df

,Name,Year,Pitch,Team,Hand,#,MPH,Inches of Drop,vs Avg,% vs Avg,...,xBA,xSLG,xwOBA,Hard Hit %,MPH,Active Spin %,Total Movement (In.),Spin-Based,Observed,Deviation
0,Brandon Pfaadt,2023,Sinker,AZ,R,160,93.0,24.4,0.9,4,...,.273,.506,.361,36.0,93.0,83,17.4,1:15,2:15,-60
1,Brandon Pfaadt,2023,Changeup,AZ,R,195,86.7,33.2,0.9,3,...,.369,.604,.419,54.3,86.7,86,16.7,1:45,2:45,-60
2,Brandon Pfaadt,2023,4-Seam Fastball,AZ,R,731,93.7,15.7,-0.2,-1,...,.285,.540,.370,49.6,93.7,77,16.2,1:15,12:45,30
3,Brandon Pfaadt,2023,Sweeper,AZ,R,435,83.8,33.0,-4.6,-12,...,.180,.317,.245,36.0,83.8,43,13.5,8:45,9:45,-60
4,Brandon Pfaadt,2023,Curveball,AZ,R,89,78.8,51.9,-2.1,-4,...,.186,.315,.240,25.0,78.8,76,17.4,7:45,7:45,0
